In [1]:
import scanpy as sc
import pandas as pd
import random
import numpy as np
import matplotlib as plt
import gc 
import ctypes
import scvi

/home/biolab/miniconda3/lib/python3.13/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
  warnings.warn(deprecation_msg, DeprecationWarning)


In [2]:
adata = sc.read_h5ad('../data/luca_query_reannotated.h5ad')

In [3]:
adata.obs

,sample,uicc_stage,ever_smoker,age,donor_id,origin,dataset,ann_fine,cell_type_predicted,doublet_status,...,IFN_TAMs,Reg_TAMs,Inflam_TAMs,LA_TAMs,Angio_TAMs,RTM_TAMs,Prolif_TAMs,Subtype,Projection_CellType,ident
001C_AAACCTGTCAACACCA-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.353933,0.571664,0.298638,0.541069,0.427279,0.565822,0.401695,Reg_TAMs,RTM_TAMs,local
001C_AAACGGGAGGCTCATT-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage,Macrophage,singlet,...,0.477315,0.451494,0.370148,0.492239,0.537751,0.511095,0.348617,Angio_TAMs,LA_TAMs,local
001C_AAACGGGGTATAATGG-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.387389,0.497444,0.282365,0.403790,0.443018,0.548354,0.328568,RTM_TAMs,RTM_TAMs,local
001C_AAACGGGTCACAAACC-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage,Macrophage,singlet,...,0.333177,0.592321,0.262999,0.594631,0.444907,0.450925,0.406192,LA_TAMs,LA_TAMs,local
001C_AAAGATGAGTGCTGCC-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.358754,0.648105,0.282604,0.482257,0.408499,0.454457,0.291259,Reg_TAMs,RTM_TAMs,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAAGCTGTCTA-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.551482,0.640531,0.489101,0.623875,0.525708,0.514441,0.527691,Reg_TAMs,IFN_TAMs,local
TTTGTCAAGGATATAC-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.492014,0.699073,0.539889,0.612688,0.608443,0.551273,0.460248,Reg_TAMs,Reg_TAMs,local
TTTGTCAAGTACGCGA-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.470449,0.588994,0.611164,0.427515,0.471975,0.431026,0.455491,Inflam_TAMs,Inflam_TAMs,local
TTTGTCACATCTATGG-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.515349,0.589936,0.633147,0.577088,0.550079,0.487074,0.425062,Inflam_TAMs,LA_TAMs,local


In [4]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [5]:
import gc 
import ctypes

In [6]:
def make_pseudobulk(adata, sample_colname, cell_type_colname, metacols):
    '''
    To create an an data with pseudo-bulk gene expression profiles for the samples and cell_types given.
    The function also carries metadata columns from the obs. metacols: as a list of col names.
    This is how to use:
        ps_adata = make_pseudobulk(hlca, sample_colname = 'sample', cell_type_colname = 'cell_type', metacols= ['patient', 'disease', 'tissue', 'study', 'sex'])
    '''
    ps_list = []
    for s in adata.obs[sample_colname].unique():
        adata_sub = adata[adata.obs[sample_colname] == s]
        
        for ct in adata_sub.obs[cell_type_colname].unique():
            adata_sub_ct = adata_sub[adata_sub.obs[cell_type_colname] == ct]
            
            adata_rep = sc.AnnData(X = adata_sub_ct.X.sum(axis = 0),
                                   var = adata_sub_ct.var[[]])
            samp_name=adata_sub.obs[sample_colname].iloc[0]
            cell= adata_sub_ct.obs[cell_type_colname].iloc[0]
            adata_rep.obs_names = [f"{samp_name}{cell}"]
            adata_rep.obs[cell_type_colname] = adata_sub_ct.obs[cell_type_colname].iloc[0]
            adata_rep.obs[metacols] = adata_sub_ct.obs[metacols].iloc[0]
            ps_list.append(adata_rep)

    ps_adata = sc.concat(ps_list)
    
    return ps_adata


In [13]:
import pandas as pd
import scipy.sparse

def savePBdata2(adata, filename):
    # Convert counts to dense matrix if it's sparse
    if scipy.sparse.issparse(adata.X):
        counts = adata.X.toarray()
    else:
        counts = adata.X

    # Convert to DataFrame
    counts_df = pd.DataFrame(counts, index=adata.obs_names, columns=adata.var_names)

    # (Optional) Convert to integer
    counts_df = counts_df.astype(int, errors='ignore')

    # Save AnnData object
    adata.write(filename)
    
    # Save counts as CSV for inspection (optional)
    counts_df.to_csv(filename.replace(".h5ad", "_counts.csv"))

    print(f"Saved AnnData to {filename} and count matrix to CSV.")


In [8]:
ps_adata_macs = make_pseudobulk(adata, sample_colname = 'sample', cell_type_colname = 'Projection_CellType', metacols= ['assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage'])

/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use 

In [14]:
savePBdata2(ps_adata_macs, filename="../data/ps_adata_macs.h5ad")


Saved AnnData to ../data/ps_adata_macs.h5ad and count matrix to CSV.


In [ ]:
with open('../data/Homo_sapiens.GRCh38.104.gtf') as f:
    gtf = list(f)

#prep the gtf file
gtf = [x for x in gtf if not x.startswith('#')]
gtf = [x for x in gtf if 'gene_id "' in x and 'gene_name "' in x]
gtf = list(map(lambda x: (x.split('gene_id "')[1].split('"')[0], x.split('gene_name "')[1].split('"')[0]), gtf))

In [10]:
ctyps = ps_adata_macs.obs['Projection_CellType'][~ps_adata_macs.obs['Projection_CellType'].isin(['Int.Node.3', 'Int.Node.4', 'Int.Node.5'])].unique()
ctyps

array(['RTM_TAMs', 'LA_TAMs', 'Prolif_TAMs', 'IFN_TAMs', 'Reg_TAMs',
       'Angio_TAMs', 'Inflam_TAMs'], dtype=object)

In [ ]:
#sub_canc = ps_adata_macs[ps_adata_macs.obs['cell_type'] == 'malignant cell' ]
df = pd.DataFrame()

for c in ctyps:
    sub_c = ps_adata_macs[ps_adata_macs.obs['Projection_CellType'] == c ]
    sub_c.obs['Contrast'] = c
    pbs = [sub_c]
    print(c)
    sub_ct = ps_adata_macs[ps_adata_macs.obs['Projection_CellType'] != c ]
    sub_ct.obs['Contrast'] = 'others'
    pbs.append(sub_ct)
    pb = sc.concat(pbs)
    print(pb)
    counts= pd.DataFrame(pb.X, columns = pb.var_names)
    counts = counts.astype(int, errors='ignore')
    #To get rid off all the 0s in count matrix (to prevent problems)
    counts=counts+1
    dds = DeseqDataSet(counts = counts, metadata=pb.obs, design_factors = ["Contrast", "assay"])
    sc.pp.filter_genes(dds, min_cells = 10)
    dds.deseq2()
    stat_res = DeseqStats(dds, contrast=('Contrast', c, 'others'))
    stat_res.summary()
    de  = stat_res.results_df
    de = de.sort_values('log2FoldChange', ascending = False)
        
    #get rownames aka gene symbols
    row_names = de.index.tolist()
    #row_names
    #add rownames as a new column 
    de['gene_symbols'] = row_names
    gtf=dict(gtf)
    de['gene_name'] = de['gene_symbols'].map(gtf)
    de = de[(de['padj'] < 0.05) & (de['log2FoldChange'] > 1.0) ]
    de['SubType'] = c
    de['inContrastTo'] = 'others'
    df = pd.concat([df, de])
    #save deg list
    df.to_csv('../results/macs_PB-DEGs.csv', index=True)

/tmp/ipykernel_2197584/1457552072.py:6: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  sub_c.obs['Contrast'] = c
/tmp/ipykernel_2197584/1457552072.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  sub_ct.obs['Contrast'] = 'others'


RTM_TAMs
AnnData object with n_obs × n_vars = 2890 × 17811
    obs: 'Projection_CellType', 'assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage', 'Contrast'


/tmp/ipykernel_2197584/1457552072.py:18: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(counts = counts, metadata=pb.obs, design_factors = ["Contrast", "assay"])
Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 2.59 seconds.

/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2197584) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2197584) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2197584) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2197584) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.1

...

/tmp/ipykernel_2197584/1457552072.py:18: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(counts = counts, metadata=pb.obs, design_factors = ["Contrast", "assay"])
Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 2.51 seconds.

Fitting dispersions...
... done in 5.86 seconds.

Fitting dispersion trend curve...
/home/biolab/miniconda3/lib/python3.13/site-packages/pydeseq2/dds.py:804: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
... done in 0.28 seconds.

Fitting MAP dispersions...
... done in 5.43 seconds.

Fitting LFCs...
... done in 7.70 seconds.

Calculating cook's distance...
... done in 4.67 seconds.

Replacing 141 outlier genes.

Fitting dispersions...
... done in 0.37 seconds.

Fitting MAP dispersions...
... done in 0.27 seconds.

Fitting LFCs...
... done in 0.31 seconds.

Running Wald tests...


Log2 fold change & Wald test p-value: Contrast Inflam_TAMs vs others
                  baseMean  log2FoldChange     lfcSE      stat        pvalue  \
ENSG00000121410   3.770136       -0.196659  0.075473 -2.605672  9.169424e-03   
ENSG00000268895   1.432926        0.049937  0.082110  0.608170  5.430748e-01   
ENSG00000175899  14.883531       -0.820015  0.114274 -7.175863  7.185245e-13   
ENSG00000245105   1.178039        0.351399  0.089818  3.912363  9.139720e-05   
ENSG00000166535   1.172041        0.295379  0.092078  3.207910  1.337033e-03   
...                    ...             ...       ...       ...           ...   
ENSG00000070476   2.052263       -0.331311  0.083579 -3.964057  7.368670e-05   
ENSG00000203995   1.158362        0.284770  0.092993  3.062256  2.196752e-03   
ENSG00000162378   3.316611       -0.508953  0.077694 -6.550773  5.724019e-11   
ENSG00000159840  13.447196       -0.413137  0.089988 -4.591025  4.410747e-06   
ENSG00000074755   3.159791       -0.426625  0.08155

... done in 5.87 seconds.

